In [1]:
import os
import re

import requests
from dotenv import load_dotenv
from tqdm.auto import tqdm

In [2]:
load_dotenv()
JINA_READER_API_KEY = os.getenv("JINA_READER_API_KEY")

In [3]:
with open("links.txt") as f:
    links = [i.strip() for i in f.readlines()]

In [4]:
text = []

In [5]:
def clean_text(text):
    """
    Remove Markdown links and images from text.

    Args:
        text (str): Input text containing Markdown links and images

    Returns:
        str: Cleaned text with links and images removed
    """
    # Remove Markdown image syntax
    text = re.sub(r"!\[.*?\]\(.*?\)", "", text)

    # Remove Markdown link syntax while keeping the link text
    text = re.sub(r"\[(.*?)\]\(.*?\)", r"\1", text)

    # Remove extra blank lines
    text = re.sub(r"\n\s*\n", "\n\n", text)

    # Strip extra whitespace
    text = text.strip()

    return text

In [6]:
headers = {"Authorization": f"Bearer {JINA_READER_API_KEY}"}

for link in tqdm(links):
    response = requests.get(f"https://r.jina.ai/{link}", headers=headers)
    text.append(f"<{link}>\n\n{clean_text(response.text)}\n\n</{link}>")

100%|██████████| 44/44 [03:22<00:00,  4.60s/it]


In [7]:
with open("extracted_content.md", "w") as f:
    f.write("\n\n------------------------------------\n\n".join(text))